In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


She said that training has labels; test does not have labels.  Let's look and see.

In [40]:
train_raw = pd.read_csv('/kaggle/input/titanic/train.csv')
df = train_raw.copy()

In [3]:
train_raw

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [256]:
test_raw = pd.read_csv('/kaggle/input/titanic/test.csv')

In [5]:
test_raw

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


## Overall Plan
1. Quick survey of every column
2. Make a simple naive neural model, see how it performs
3. Try with XGBoost
4. Compare with NN model with embeddings
5. See how XGBoost performs with the embeddings I created

## Data Preprocessing

## Column Review
Here we'll do a quick review of all of the columns to determine how we want to handle each column.

### Pclass

In [41]:
df['Pclass'].unique()

array([3, 1, 2])

No missing data or anything funny going on here. We'll just use this one as a categorical variable and we're good.


## Name

In [14]:
list(df['Name'])

['Braund, Mr. Owen Harris',
 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
 'Heikkinen, Miss. Laina',
 'Futrelle, Mrs. Jacques Heath (Lily May Peel)',
 'Allen, Mr. William Henry',
 'Moran, Mr. James',
 'McCarthy, Mr. Timothy J',
 'Palsson, Master. Gosta Leonard',
 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)',
 'Nasser, Mrs. Nicholas (Adele Achem)',
 'Sandstrom, Miss. Marguerite Rut',
 'Bonnell, Miss. Elizabeth',
 'Saundercock, Mr. William Henry',
 'Andersson, Mr. Anders Johan',
 'Vestrom, Miss. Hulda Amanda Adolfina',
 'Hewlett, Mrs. (Mary D Kingcome) ',
 'Rice, Master. Eugene',
 'Williams, Mr. Charles Eugene',
 'Vander Planke, Mrs. Julius (Emelia Maria Vandemoortele)',
 'Masselmani, Mrs. Fatima',
 'Fynney, Mr. Joseph J',
 'Beesley, Mr. Lawrence',
 'McGowan, Miss. Anna "Annie"',
 'Sloper, Mr. William Thompson',
 'Palsson, Miss. Torborg Danira',
 'Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)',
 'Emir, Mr. Farred Chehab',
 'Fortune, Mr. Charles Alexander',
 '

There is a little bit of extra information in here.  For the women, we can see if they are married or not.  Also some people have nicknames; it's possible that can correlate with survival but I'm going to ignore that for now.

## Age

In [24]:
df['Age'].unique()

array([22.  , 38.  , 26.  , 35.  ,   nan, 54.  ,  2.  , 27.  , 14.  ,
        4.  , 58.  , 20.  , 39.  , 55.  , 31.  , 34.  , 15.  , 28.  ,
        8.  , 19.  , 40.  , 66.  , 42.  , 21.  , 18.  ,  3.  ,  7.  ,
       49.  , 29.  , 65.  , 28.5 ,  5.  , 11.  , 45.  , 17.  , 32.  ,
       16.  , 25.  ,  0.83, 30.  , 33.  , 23.  , 24.  , 46.  , 59.  ,
       71.  , 37.  , 47.  , 14.5 , 70.5 , 32.5 , 12.  ,  9.  , 36.5 ,
       51.  , 55.5 , 40.5 , 44.  ,  1.  , 61.  , 56.  , 50.  , 36.  ,
       45.5 , 20.5 , 62.  , 41.  , 52.  , 63.  , 23.5 ,  0.92, 43.  ,
       60.  , 10.  , 64.  , 13.  , 48.  ,  0.75, 53.  , 57.  , 80.  ,
       70.  , 24.5 ,  6.  ,  0.67, 30.5 ,  0.42, 34.5 , 74.  ])

This column is going to require some special handling.
* For infants, age is fractional
* If age is unknown, then it's estimated as xx.5
* There are some unknown ages (nan).

In [92]:
def age_processor(df):
    df['age_known'] = (df.Age < 1) | (df.Age == np.floor(df.Age))
    # People with an age > 1 and a fraction of 0.5, their age is estimated.
    df['adjusted_age'] = np.where(df.age_known, df.Age, np.floor(df.Age))
    return df

In [125]:
processors = [
    age_processor
]
def prep_data():
    df = pd.read_csv('/kaggle/input/titanic/train.csv')
    for processor in processors:
        df = processor(df)
    return df

In [126]:
prep_data()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_known,adjusted_age
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,True,22.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,True,38.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,True,26.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,True,35.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,True,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,True,27.0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,True,19.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,False,NaN
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,True,26.0


## SibSp, Parch
Looks ok; not thinking of any changes I want to make to these columns right now.

In [97]:
df['SibSp'].unique()

array([1, 0, 3, 4, 2, 5, 8])

In [98]:
df['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6])

## Ticket

In [99]:
df['Ticket'].unique()

array(['A/5 21171', 'PC 17599', 'STON/O2. 3101282', '113803', '373450',
       '330877', '17463', '349909', '347742', '237736', 'PP 9549',
       '113783', 'A/5. 2151', '347082', '350406', '248706', '382652',
       '244373', '345763', '2649', '239865', '248698', '330923', '113788',
       '347077', '2631', '19950', '330959', '349216', 'PC 17601',
       'PC 17569', '335677', 'C.A. 24579', 'PC 17604', '113789', '2677',
       'A./5. 2152', '345764', '2651', '7546', '11668', '349253',
       'SC/Paris 2123', '330958', 'S.C./A.4. 23567', '370371', '14311',
       '2662', '349237', '3101295', 'A/4. 39886', 'PC 17572', '2926',
       '113509', '19947', 'C.A. 31026', '2697', 'C.A. 34651', 'CA 2144',
       '2669', '113572', '36973', '347088', 'PC 17605', '2661',
       'C.A. 29395', 'S.P. 3464', '3101281', '315151', 'C.A. 33111',
       'S.O.C. 14879', '2680', '1601', '348123', '349208', '374746',
       '248738', '364516', '345767', '345779', '330932', '113059',
       'SO/C 14885', '31012

There could be some useful data in here.  Let's find the unique tokens.

In [119]:
tokens = set()
numerics = set()
for t in df['Ticket'].unique():
    t = t.upper()
    t = t.replace('.','')
    parts = t.split(' ')
    for part in parts:
        if part.isnumeric():
            numerics.add(int(part))
            continue
        tokens.add(part)

In [120]:
sorted(tokens)

['A/4',
 'A/5',
 'A/S',
 'A4',
 'A5',
 'BASLE',
 'C',
 'CA',
 'CA/SOTON',
 'FA',
 'FC',
 'FCC',
 'LINE',
 'P/PP',
 'PC',
 'PP',
 'SC',
 'SC/A4',
 'SC/AH',
 'SC/PARIS',
 'SCO/W',
 'SO/C',
 'SO/PP',
 'SOC',
 'SOP',
 'SOTON/O2',
 'SOTON/OQ',
 'SP',
 'STON/O',
 'STON/O2',
 'SW/PP',
 'W/C',
 'WE/P',
 'WEP']

In [116]:
sorted(numerics)

[3,
 541,
 693,
 695,
 751,
 752,
 851,
 1166,
 1585,
 1601,
 1748,
 2003,
 2079,
 2123,
 2131,
 2133,
 2144,
 2146,
 2149,
 2151,
 2152,
 2163,
 2167,
 2223,
 2314,
 2315,
 2343,
 2466,
 2620,
 2623,
 2624,
 2625,
 2626,
 2627,
 2628,
 2629,
 2631,
 2641,
 2647,
 2648,
 2649,
 2650,
 2651,
 2653,
 2659,
 2661,
 2662,
 2663,
 2664,
 2665,
 2666,
 2667,
 2668,
 2669,
 2671,
 2672,
 2673,
 2674,
 2677,
 2678,
 2680,
 2683,
 2685,
 2686,
 2687,
 2689,
 2690,
 2691,
 2693,
 2694,
 2695,
 2697,
 2699,
 2700,
 2816,
 2817,
 2908,
 2926,
 3085,
 3235,
 3336,
 3337,
 3381,
 3411,
 3460,
 3464,
 3474,
 3536,
 3540,
 3594,
 3902,
 4001,
 4133,
 4134,
 4135,
 4136,
 4137,
 4138,
 4348,
 4579,
 5547,
 5727,
 5734,
 5735,
 6212,
 6563,
 6607,
 6608,
 6609,
 7075,
 7076,
 7077,
 7267,
 7534,
 7540,
 7545,
 7546,
 7552,
 7553,
 7598,
 8471,
 8475,
 9234,
 9549,
 10482,
 11206,
 11668,
 11751,
 11752,
 11753,
 11755,
 11765,
 11767,
 11769,
 11771,
 11774,
 11813,
 11967,
 12233,
 12460,
 12749,
 1275

Seems like there could be something useful in those alpha tokens in the tickets.  Also it seems interesting that there are some large groups of tickets in the ticket numbers.  This could also be useful information.

In [124]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_known,adjusted_age
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,True,22.0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,True,38.0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,True,26.0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,True,35.0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,True,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,True,27.0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,True,19.0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,False,NaN
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,True,26.0


In [292]:
def ticket_tokens(ticket_str):
    t = ticket_str.upper()
    for remove in ['.','/']:
        t = t.replace(remove,'')
    for token in t.split(' '):
        yield token

def ticket_code(ticket_str):
    for token in ticket_tokens(ticket_str):
        if not token.isnumeric():
            return token
    return ''

def ticket_numeric(ticket_str):
    for token in ticket_tokens(ticket_str):
        if token.isnumeric():
            return int(token)
    return 0
        
def ticket_processor(df):
    df['ticket_code'] = df.Ticket.apply(ticket_code)
    df['ticket_numeric'] = df.Ticket.apply(ticket_numeric)
    return df
    

In [201]:
ticket = 'PC 17599'
list(ticket_tokens(ticket))

['PC', '17599']

In [202]:
ticket_code(ticket)

'PC'

In [203]:
ticket_numeric(ticket)

17599

In [207]:
df = ticket_processor(train_raw.copy())


In [208]:
sorted(list(df['ticket_code'].unique()))

['',
 'A4',
 'A5',
 'AS',
 'C',
 'CA',
 'CASOTON',
 'FA',
 'FC',
 'FCC',
 'LINE',
 'PC',
 'PP',
 'PPP',
 'SC',
 'SCA4',
 'SCAH',
 'SCOW',
 'SCPARIS',
 'SOC',
 'SOP',
 'SOPP',
 'SOTONO2',
 'SOTONOQ',
 'SP',
 'STONO',
 'STONO2',
 'SWPP',
 'WC',
 'WEP']

There might be more room to play with this; it's possible that things like `WE/P` and `WEP` are really the same thing.  And maybe there's some correlation of meaning between `SC/A4`, `SC`, and `A4`.  In fact, let's go ahead and integrate those by dropping slashes.

In [209]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,ticket_code,ticket_numeric
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,A5,21171
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,PC,17599
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,STONO2,3101282
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,,113803
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,,373450
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,NaN,S,,211536
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,B42,S,,112053
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,23.4500,NaN,S,WC,6607
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C148,C,,111369


## Fare

In [212]:
sorted(list(df['Fare'].unique()))

[0.0,
 4.0125,
 5.0,
 6.2375,
 6.4375,
 6.45,
 6.4958,
 6.75,
 6.8583,
 6.95,
 6.975,
 7.0458,
 7.05,
 7.0542,
 7.125,
 7.1417,
 7.225,
 7.2292,
 7.25,
 7.3125,
 7.4958,
 7.5208,
 7.55,
 7.6292,
 7.65,
 7.725,
 7.7292,
 7.7333,
 7.7375,
 7.7417,
 7.75,
 7.775,
 7.7875,
 7.7958,
 7.8,
 7.8292,
 7.8542,
 7.875,
 7.8792,
 7.8875,
 7.8958,
 7.925,
 8.0292,
 8.05,
 8.1125,
 8.1375,
 8.1583,
 8.3,
 8.3625,
 8.4042,
 8.4333,
 8.4583,
 8.5167,
 8.6542,
 8.6625,
 8.6833,
 8.7125,
 8.85,
 9.0,
 9.2167,
 9.225,
 9.35,
 9.475,
 9.4833,
 9.5,
 9.5875,
 9.825,
 9.8375,
 9.8417,
 9.8458,
 10.1708,
 10.4625,
 10.5,
 10.5167,
 11.1333,
 11.2417,
 11.5,
 12.0,
 12.275,
 12.2875,
 12.35,
 12.475,
 12.525,
 12.65,
 12.875,
 13.0,
 13.4167,
 13.5,
 13.7917,
 13.8583,
 13.8625,
 14.0,
 14.1083,
 14.4,
 14.4542,
 14.4583,
 14.5,
 15.0,
 15.0458,
 15.05,
 15.1,
 15.2458,
 15.5,
 15.55,
 15.7417,
 15.75,
 15.85,
 15.9,
 16.0,
 16.1,
 16.7,
 17.4,
 17.8,
 18.0,
 18.75,
 18.7875,
 19.2583,
 19.5,
 19.9667,
 20.2

Looks ok.  I'm not sure why some of them are zero.  The fractional numbers suggests maybe there is currency conversion involved.  I don't see any NaN or things that need processing.

## Cabin

In [227]:
def cabin_tokens(s):
    if pd.isna(s):
        return None
    return s
    
for s in df.Cabin:
    t = cabin_tokens(s)
    if not t:
        continue
    print(t)

C85
C123
E46
G6
C103
D56
A6
C23 C25 C27
B78
D33
B30
C52
B28
C83
F33
F G73
C23 C25 C27
E31
A5
D10 D12
D26
C110
B58 B60
E101
D26
F E69
D47
C123
B86
F2
C2
E33
B19
A7
C49
F4
A32
F2
B4
B80
G6
A31
D36
D15
C93
C83
C78
D35
G6
C87
B77
E67
B94
C125
C99
C118
D7
A19
B49
D
C22 C26
C106
B58 B60
E101
C22 C26
C65
E36
C54
B57 B59 B63 B66
C7
E34
C32
D
B18
C124
C91
C2
E40
T
F2
C23 C25 C27
F33
C128
E33
D37
B35
E50
C82
B96 B98
D36
G6
C78
E10
C52
E44
B96 B98
C23 C25 C27
A34
C104
C111
C92
E38
D21
E12
E63
D
A14
B49
C93
B37
C30
D20
C22 C26
B79
C65
E25
D46
F33
B73
B18
C95
B38
B39
B22
C86
C70
A16
E67
C101
E25
E44
C68
A10
E68
B41
D20
A20
C125
F4
D19
D50
D9
A23
B50
B35
D33
A26
D48
E58
C126
B71
B51 B53 B55
D49
B5
B20
C68
F G63
C62 C64
E24
E24
C90
C124
C126
F G73
C45
E101
E8
B5
B101
D45
C46
B57 B59 B63 B66
B22
D30
E121
B77
B96 B98
D11
E77
F38
B3
B20
D6
B82 B84
D17
B96 B98
A36
E8
B102
B69
E121
B28
E49
C47
C92
D28
E17
D17
A24
D35
B51 B53 B55
C50
B42
C148


Some people have multiple cabins, apparently?  Plus there are some other interesting things in here.

In [234]:
df[df.Cabin.isin(['B57 B59 B63 B66', 'F G73'])].sort_values(['Cabin'])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,ticket_code,ticket_numeric
311,312,1,1,"Ryerson, Miss. Emily Borie",female,18.0,2,2,262.375,B57 B59 B63 B66,C,PC,17608
742,743,1,1,"Ryerson, Miss. Susan Parker ""Suzette""",female,21.0,2,2,262.375,B57 B59 B63 B66,C,PC,17608
75,76,0,3,"Moen, Mr. Sigurd Hansen",male,25.0,0,0,7.650,F G73,S,,348123
715,716,0,3,"Soholt, Mr. Peter Andreas Lauritz Andersen",male,19.0,0,0,7.650,F G73,S,,348124


For now, I'll divide the cabin information into 2 fields:
* A single code for the cabin section, `A`,`B`, etc. Select the first code from a cabin with a room number.  So for `F G73` we will return `G`.
* A single numeric for the room number.  If there are multiple, I'll use the first one.

In [287]:
def room_number(cabin):
    if pd.isna(cabin):
        return 0
    for part in cabin.split(' '):
        if len(part) <=1: # Skip single letter things
            continue
        return int(part[1:])
    return 0

def room_known(cabin):
    return (room_number(cabin) > 0)

def room_section(cabin):
    if pd.isna(cabin):
        return 'Unk'
    result = 'Unk'
    for part in cabin.split(' '):
        if len(part) <=1: # Skip single letter things
            if result == 'Unk': # Use this for the section if we haven't found something better.
                result = part
        return part[0:1]
    return result

In [288]:
test_cabins = ['B57 B58', 'F G73','A12', 'T']
[room_number(c) for c in test_cabins]

[57, 73, 12, 0]

In [289]:
[room_section(c) for c in test_cabins]

['B', 'F', 'A', 'T']

In [290]:
def cabin_processor(df):
    df['cabin_known'] = df.Cabin.apply(room_known)
    df['cabin_number'] = df.Cabin.apply(room_number)
    df['cabin_section'] = df.Cabin.apply(room_section)
    return df

In [296]:
df = cabin_processor(test_raw.copy())
df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,cabin_known,cabin_number,cabin_section
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,False,0,Unk
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,False,0,Unk
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,False,0,Unk
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,False,0,Unk
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,False,0,Unk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,False,0,Unk
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,True,105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,False,0,Unk
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,False,0,Unk


In [297]:
df[df.cabin_section != 'Unk']

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,cabin_known,cabin_number,cabin_section
12,904,1,"Snyder, Mrs. John Pillsbury (Nelle Stevenson)",female,23.0,1,0,21228,82.2667,B45,S,True,45,B
14,906,1,"Chaffee, Mrs. Herbert Fuller (Carrie Constance...",female,47.0,1,0,W.E.P. 5734,61.1750,E31,S,True,31,E
24,916,1,"Ryerson, Mrs. Arthur Larned (Emily Maria Borie)",female,48.0,1,3,PC 17608,262.3750,B57 B59 B63 B66,C,True,57,B
26,918,1,"Ostby, Miss. Helene Ragnhild",female,22.0,0,1,113509,61.9792,B36,C,True,36,B
28,920,1,"Brady, Mr. John Bertram",male,41.0,0,0,113054,30.5000,A21,S,True,21,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,1296,1,"Frauenthal, Mr. Isaac Gerald",male,43.0,1,0,17765,27.7208,D40,C,True,40,D
405,1297,2,"Nourney, Mr. Alfred (Baron von Drachstedt"")""",male,20.0,0,0,SC/PARIS 2166,13.8625,D38,C,True,38,D
407,1299,1,"Widener, Mr. George Dunton",male,50.0,1,1,113503,211.5000,C80,C,True,80,C
411,1303,1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37.0,1,0,19928,90.0000,C78,Q,True,78,C


## Embarked

In [298]:
df.Embarked.unique()

array(['Q', 'S', 'C'], dtype=object)

It looks like that's all for the data preprocessing.  Let's bring all the processors together.

In [302]:
processors = [
    age_processor,
    ticket_processor,
    cabin_processor,
]

def prep_data():
    df = pd.read_csv('/kaggle/input/titanic/train.csv')
    for processor in processors:
        df = processor(df)
    raw_data = df
    # Trim down to just the columns we'll look at.
    trimmed = df.copy().drop(columns=['Name','Age','Ticket','Cabin'])
    return raw_data, trimmed

In [303]:
raw, df = prep_data()

In [304]:
raw


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_known,adjusted_age,ticket_code,ticket_numeric,cabin_known,cabin_number,cabin_section
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,True,22.0,A5,21171,False,0,Unk
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,True,38.0,PC,17599,True,85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,True,26.0,STONO2,3101282,False,0,Unk
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,True,35.0,,113803,True,123,C
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,True,35.0,,373450,False,0,Unk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,True,27.0,,211536,False,0,Unk
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,True,19.0,,112053,True,42,B
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,False,NaN,WC,6607,False,0,Unk
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,True,26.0,,111369,True,148,C


Here is what we'll train with:

In [305]:
df

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,age_known,adjusted_age,ticket_code,ticket_numeric,cabin_known,cabin_number,cabin_section
0,1,0,3,male,1,0,7.2500,S,True,22.0,A5,21171,False,0,Unk
1,2,1,1,female,1,0,71.2833,C,True,38.0,PC,17599,True,85,C
2,3,1,3,female,0,0,7.9250,S,True,26.0,STONO2,3101282,False,0,Unk
3,4,1,1,female,1,0,53.1000,S,True,35.0,,113803,True,123,C
4,5,0,3,male,0,0,8.0500,S,True,35.0,,373450,False,0,Unk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,0,0,13.0000,S,True,27.0,,211536,False,0,Unk
887,888,1,1,female,0,0,30.0000,S,True,19.0,,112053,True,42,B
888,889,0,3,female,1,2,23.4500,S,False,NaN,WC,6607,False,0,Unk
889,890,1,1,male,0,0,30.0000,C,True,26.0,,111369,True,148,C
